In [1]:
import findspark

findspark.init()
import pyspark

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [3]:
def readDataFrameFromCloudant(database):

    cloudantdata = spark.read.format("org.apache.bahir.cloudant")\
        .option("cloudant.host",'107eaa35-216a-4dc5-9d5d-02e7e734f3ba-bluemix.cloudantnosqldb.appdomain.cloud')\
        .option("cloudant.username", 'apikey-v2-1e1faveibzoosftyog8b3w9pz8s6zmax90fwnhfwtvnf')\
        .option("cloudant.password",'0928dfec94454b6511e31685e584fad3')\
        .load(database)

    return cloudantdata

In [4]:
df = readDataFrameFromCloudant('training')
df.createOrReplaceTempView('df')
df = df.dropna()
spark.sql('select class, count(class) from df group by class').show()

Py4JJavaError: An error occurred while calling o27.load.
: java.lang.ClassNotFoundException: Failed to find data source: org.apache.bahir.cloudant. Please find packages at http://spark.apache.org/third-party-projects.html
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:689)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:743)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:266)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:240)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(Unknown Source)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(Unknown Source)
	at java.lang.reflect.Method.invoke(Unknown Source)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:28

In [ ]:
df

In [ ]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import Normalizer
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
vectorAssembler = VectorAssembler(inputCols=["Current", "Step"], outputCol="features")
lr = LogisticRegression(maxIter=1000).setLabelCol("class")
pipeline = Pipeline(stages=[vectorAssembler, lr ])
model = pipeline.fit(df)
result = model.transform(df)

In [ ]:
print('intercept = ', model.stages[1].intercept)
print('coefficients = ', model.stages[1].coefficients)

In [ ]:
binEval = MulticlassClassificationEvaluator().setMetricName("f1").setPredictionCol("prediction").setLabelCol("class")
binEval.evaluate(result)

In [ ]:
new_df = readDataFrameFromCloudant('training')
new_df.createOrReplaceTempView('new_df')
result = model.transform(new_df)
result.createOrReplaceTempView('result')
spark.sql("select Current, Step, class, prediction from result").show(50)